# The Knapsack Problem

The knapsack problem involves a knapsack with a maximum weight capacity and a set of items with a weight and a value. The goal is to maximize the total value of the items in the knapsack without exceeding the maximum weight capacity. The knapsack problem is a classic problem in the field of operations research. It is used in many applications, such as resource allocation, financial portfolio selection, and scheduling.

<img src="img/knapsack.png" alt="Knapsack" style="width:400px; height:400px;">

In this notebook, we will show how to model and solve the knapsack problem using SeaPearl.jl.

## Setup
We will begin by activating the environment and importing the necessary packages.

In [1]:
using Revise
using Pkg
Pkg.activate("../../../")
using SeaPearl

  Activating project at `c:\Users\leobo\Desktop\École\Poly\SeaPearl\SeaPearlZoo.jl`


## Problem formulation

The instances have the following format:
```
4 11
8 4
10 5
15 8
4 3
```
The instances have the following structure: the first line contains the number of items and the capacity of the knapsack; the following lines contain the value and weight of items. For example, this instance contains 4 items, the knapsack capacity is 11, the first item has a value of 8 and a weight of 4, the second item has a value of 10 and a weight of 5, and so on.

## Parsing the Instances

We will now build utilities to parse the instances.

In [2]:
"""Item
   item in the knapsack problem
"""
struct Item
    id      :: Int
    value   :: Int
    weight  :: Int
end

"""KnapsackInputData
   input data for the knapsack problem
"""
struct KnapsackInputData
    items               :: AbstractArray{Union{Item, Nothing}}
    sortedItems         :: AbstractArray{Union{Item, Nothing}}
    numberOfItems       :: Int
    capacity            :: Int
end

"""parseKnapsackInput!
   parse the input data for the knapsack problem
"""
function parseKnapsackInput!(filename::String)
    input_data = nothing
    open(filename, "r") do openedFile
        input_data = read(openedFile, String)
    end
    lines = split(input_data, '\n')
    firstLine = split(lines[1], ' ')
    numberOfItems = parse(Int, firstLine[1])
    capacity = parse(Int, firstLine[2])
    items = Array{Union{Item}}(undef, numberOfItems);

    @assert numberOfItems + 2 <= length(lines)

    for i in 1:numberOfItems
        itemArray = split(lines[i+1], ' ')
        item = Item(i, parse(Int, itemArray[1]), parse(Int, itemArray[2]))
        items[i] = item
    end

    return KnapsackInputData(items, Item[], numberOfItems, capacity)
end

parseKnapsackInput!

In [4]:
knapsack_instance = parseKnapsackInput!("./data/ks_4_0")

KnapsackInputData(Union{Nothing, Item}[Item(1, 8, 4), Item(2, 10, 5), Item(3, 15, 8), Item(4, 4, 3)], Union{Nothing, Item}[], 4, 11)

## Modeling the Problem

We will now build a model for the problem

In [6]:
sorted_relative_value = sortperm(knapsack_instance.items; by=(x) -> x.value / x.weight, rev=true) # sort items by relative value
trailer = SeaPearl.Trailer()
model = SeaPearl.CPModel(trailer)
num_items = knapsack_instance.numberOfItems
# =========VARIABLES=========
# add variables representing item selection
item_selection = SeaPearl.IntVar[]
for i in 1:num_items
    push!(item_selection, SeaPearl.IntVar(0, 1, "item[" * string(i) * "]", trailer))
    SeaPearl.addVariable!(model, last(item_selection))
end
# add variables representing the weight of the item in the knapsack. If the item is not selected, the variable associated to this item will have 
# a value of zero
item_weight_in_knapsack = SeaPearl.AbstractIntVar[]
max_weight = 0
for i in 1:num_items
    current_item_index = sorted_relative_value[i]
    current_item = knapsack_instance.items[current_item_index]
    current_item_weight_in_knapsack = SeaPearl.IntVarViewMul(item_selection[i], current_item.weight, "weight_item[" * string(i) * "]")
    push!(item_weight_in_knapsack, current_item_weight_in_knapsack)
    max_weight += current_item.weight
end
total_weight = SeaPearl.IntVar(0, max_weight, "total_weight", trailer) # total weight of items in knapsack
negative_total_weight = SeaPearl.IntVarViewOpposite(total_weight, "-total_weight") # -1 * total_weight; necessary because the solver can only minimize objectives
SeaPearl.addVariable!(model, total_weight)
SeaPearl.addVariable!(model, negative_total_weight)
push!(item_weight_in_knapsack, negative_total_weight) # added to array to later add a constraint that the sum of the array's elements == 0

item_value_in_knapsack = SeaPearl.AbstractIntVar[]
max_value = 0
for i in 1:num_items
    current_item_index = sorted_relative_value[i]
    current_item = knapsack_instance.items[current_item_index]
    current_item_value = SeaPearl.IntVarViewMul(item_selection[i], current_item.value, "value_item[" * string(i) * "]")
    push!(item_value_in_knapsack, current_item_value)
    max_value += current_item.value
end
total_value = SeaPearl.IntVar(-max_value, 0, "totalValue", trailer)
SeaPearl.addVariable!(model, total_value)
push!(item_value_in_knapsack, total_value)

# =========CONSTRAINTS=========

# consistency of negative weight in knapsack
weight_sums_to_zero = SeaPearl.SumToZero(item_weight_in_knapsack, trailer)
push!(model.constraints, weight_sums_to_zero)
# weight below max capacity
weight_constraint = SeaPearl.LessOrEqualConstant(total_weight, knapsack_instance.capacity, trailer)
push!(model.constraints, weight_constraint)
# consistency of negative value variable in knapsack
valueEquality = SeaPearl.SumToZero(item_value_in_knapsack, trailer)
push!(model.constraints, valueEquality)

# =========OBJECTIVE=========
SeaPearl.addObjective!(model, total_value)

Int64[]

## Variable Selection Heuristic

For this problem, we will build a variable selection heuristic. It will choose the first variable that is not fixed.

In [7]:
struct VariableSelection{TakeObjective} <: SeaPearl.AbstractVariableSelection{TakeObjective} end
VariableSelection(; take_objective=true) = VariableSelection{take_objective}()

"""(::VariableSelection{true})(model::SeaPearl.CPModel)::SeaPearl.AbstractIntVar

"""
function (::VariableSelection{true})(model::SeaPearl.CPModel)::SeaPearl.AbstractIntVar
    i = 1
    while SeaPearl.isbound(model.variables["item["*string(i)*"]"])
        i += 1
    end
    return model.variables["item["*string(i)*"]"]
end

VariableSelection

## Solving the Problem

The model is built; let's solve it!

In [8]:
SeaPearl.solve!(model; variableHeuristic=VariableSelection{true}())

:Optimal

## Visualizing the Solution

Let's visualize the solution obtained

In [11]:
function get_best_solution(model::SeaPearl.CPModel)
    best_solution = nothing
    best_objective = Inf
    for solution in model.statistics.solutions
        if !isnothing(solution) && solution["totalValue"] < best_objective
            best_solution = solution
            best_objective = solution["totalValue"]
        end
    end
    return best_solution
end
best_solution = get_best_solution(model)

Dict{String, Union{Bool, Int64, Set{Int64}}} with 7 entries:
  "item[3]"       => 1
  "item[4]"       => 1
  "item[2]"       => 0
  "totalValue"    => -19
  "total_weight"  => 11
  "item[1]"       => 0
  "-total_weight" => -11